In [1]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
from datetime import datetime
# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

financial = pd.read_csv("../../data/processed_03/financial-prices-ratios.csv", index_col=0)

# Clean Ratios

In [2]:
columns_ratios = columns['columns_ratios']
columns_ratios_all = columns['column_id'] + columns['columns_ratios']
ratios_df = financial[columns_ratios_all]

In [3]:
for column in columns_ratios:
    alpha = 0.1
    top_percentile = 1 - (alpha/2)
    low_percentile = alpha/2

    # Calculate the alpha percentile value
    top_percentile_value = ratios_df[column].quantile(top_percentile)
    low_percentile_value = ratios_df[column].quantile(low_percentile)

    # Replace outliers with percentiles value
    ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
    ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)

C:\Users\germa\AppData\Local\Temp/ipykernel_1748/2285555845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/2285555845.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)


In [4]:
# # Replace -inf and inf with None
# ratios_df = ratios_df.replace([-np.inf, np.inf], None)

# Make sure types are correct
ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
ratios_df['PK'] = ratios_df['PK'].astype(int)
ratios_df['Date'] = pd.to_datetime(ratios_df['Date'])



C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3579048468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3579048468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df['PK'] = ratios_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3579048468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [5]:
ratios_df.describe()

,PK,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
count,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000,48627.000000
mean,24313.000000,12355.759653,8.834095,13.569448,18.309980,523.758135,173.145251,10.308200,10.308200,11.290916,5.733959,7.601875
std,14037.550107,24080.343427,10.262799,24.554199,38.179124,384.166760,645.933346,57.303896,57.303896,28.530990,8.287569,18.594565
min,0.000000,19.039855,0.521334,-48.042048,-74.842942,1.234302,-863.946600,-120.291160,-120.291160,-52.194293,0.248127,-38.231145
25%,12156.500000,345.841588,1.558262,1.828617,-2.079555,28.975671,-51.915500,-13.122266,-13.122266,-3.049934,0.915335,-0.621428
50%,24313.000000,1898.509844,3.665887,12.314333,16.013031,832.765478,1.673000,4.990149,4.990149,11.751725,2.171148,7.922922
75%,36469.500000,9296.041324,12.986392,23.065916,34.698624,832.765478,157.898500,30.763722,30.763722,22.736590,5.387872,15.667348
max,48626.000000,90207.763914,31.267600,53.672690,81.077526,832.765478,1944.739000,133.026000,133.026000,81.139495,27.130974,47.606554


In [6]:
ratios_df.head(2)

,PK,ticker,Date,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
0,0,A,2009-10-31,8728.061863,1.947793,41.761062,81.077526,53.546392,394.0,22.152441,22.152441,-52.194293,1.366450,15.007505
1,1,A,2010-10-31,11048.124516,2.029413,14.385579,19.519655,77.803694,-456.0,-24.228343,-24.228343,12.831291,1.614093,12.238335


# Clean Income Statement

In [7]:
columns_income_statement = columns['column_id'] + columns['columns_income_statement']
income_statememnt_df = financial[columns_income_statement]

In [8]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,48627.000000,45793.000000,41518.000000,45826.000000,18617.000000,46722.000000,20988.000000,41515.000000,48455.000000,43417.000000,...,1262.000000,48528.000000,8467.000000,48528.000000,43688.000000,43685.000000,48427.000000,48400.000000,4.843500e+04,4.862700e+04
mean,24313.000000,5832.876939,3847.116916,2343.218566,318.947128,1098.470992,-350.071323,1672.023152,685.326363,-89.644798,...,10.023448,455.848351,59.087751,440.615630,1004.369318,647.111197,262.128931,267.443594,-4.872049e+03,-4.853635e+03
std,14037.550107,22103.536334,16775.844067,8329.345448,1607.334002,4520.447814,2569.097082,5995.039360,3188.531066,859.365004,...,151.596240,2372.724484,1030.972710,2371.992174,4143.116705,3108.649442,2044.294423,2071.745049,7.408631e+05,7.393989e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.000000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,12156.500000,97.441000,41.613000,46.861250,6.376000,23.208500,-71.119000,51.771900,-5.686000,-52.700000,...,0.000000,-10.255575,-2.296000,-11.146725,-1.854025,-8.287000,15.067000,15.289750,-5.600000e-01,-5.500000e-01
50%,24313.000000,597.458000,327.909000,254.095500,28.751000,104.322000,-6.048500,213.485000,38.068000,-4.346000,...,0.000000,20.394500,0.000000,19.385500,76.307000,37.310000,44.782000,45.607450,6.500000e-01,6.231000e-01
75%,36469.500000,2959.238000,1841.366750,1134.842000,100.146000,481.173250,0.000000,831.522500,290.207850,0.176000,...,0.000000,182.692250,8.933500,175.632000,485.500125,299.225900,135.567500,138.400000,2.180000e+00,2.140000e+00
max,48626.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.000000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


In [9]:
# Make sure types are correct
income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
income_statememnt_df['Date'] = pd.to_datetime(income_statememnt_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_1748/800232268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/800232268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/800232268.py:4: SettingWithCopyWarning: 


In [10]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,48627.000000,45793.000000,41518.000000,45826.000000,18617.000000,46722.000000,20988.000000,41515.000000,48455.000000,43417.000000,...,1262.000000,48528.000000,8467.000000,48528.000000,43688.000000,43685.000000,48427.000000,48400.000000,4.843500e+04,4.862700e+04
mean,24313.000000,5832.876939,3847.116916,2343.218566,318.947128,1098.470992,-350.071323,1672.023152,685.326363,-89.644798,...,10.023448,455.848351,59.087751,440.615630,1004.369318,647.111197,262.128931,267.443594,-4.872049e+03,-4.853635e+03
std,14037.550107,22103.536334,16775.844067,8329.345448,1607.334002,4520.447814,2569.097082,5995.039360,3188.531066,859.365004,...,151.596240,2372.724484,1030.972710,2371.992174,4143.116705,3108.649442,2044.294423,2071.745049,7.408631e+05,7.393989e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.000000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,12156.500000,97.441000,41.613000,46.861250,6.376000,23.208500,-71.119000,51.771900,-5.686000,-52.700000,...,0.000000,-10.255575,-2.296000,-11.146725,-1.854025,-8.287000,15.067000,15.289750,-5.600000e-01,-5.500000e-01
50%,24313.000000,597.458000,327.909000,254.095500,28.751000,104.322000,-6.048500,213.485000,38.068000,-4.346000,...,0.000000,20.394500,0.000000,19.385500,76.307000,37.310000,44.782000,45.607450,6.500000e-01,6.231000e-01
75%,36469.500000,2959.238000,1841.366750,1134.842000,100.146000,481.173250,0.000000,831.522500,290.207850,0.176000,...,0.000000,182.692250,8.933500,175.632000,485.500125,299.225900,135.567500,138.400000,2.180000e+00,2.140000e+00
max,48626.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.000000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


# Clean balance sheet

In [11]:
columns_balance_sheet = columns['column_id'] + columns['columns_balance_sheet']
balance_sheet_df = financial[columns_balance_sheet]

In [12]:
# Make sure types are correct
balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
balance_sheet_df['Date'] = pd.to_datetime(balance_sheet_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_1748/19496767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/19496767.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/19496767.py:4: SettingWithCopyWarning: 
A value is trying to be s

# Clean columns_cash_flow_statement

In [13]:
columns_cash_flow_statement = columns['column_id'] + columns['columns_cash_flow_statement']
cash_flow_statement_df = financial[columns_cash_flow_statement]

In [14]:
# Make sure types are correct
cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
cash_flow_statement_df['Date'] = pd.to_datetime(cash_flow_statement_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_1748/475317979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/475317979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/475317979.py:4: SettingWith

# Clean prices

In [15]:
columns_prices = columns['column_id'] + columns['columns_prices']
prices_df = financial[columns_prices]

In [16]:
prices_df.describe()

,PK,longevity,Close,Volume,Growth -1,Growth +1,Growth +5,Growth -10,Growth -5
count,48627.000000,48627.000000,4.862700e+04,4.862700e+04,45729.000000,43247.000000,25892.000000,37168.000000,30160.000000
mean,24313.000000,16.052522,6.781296e+07,3.118435e+07,0.152241,0.131347,0.744440,0.706449,1.935583
std,14037.550107,12.458210,8.632753e+09,1.610555e+08,1.655892,1.495676,3.236733,4.763019,9.992951
min,0.000000,0.000000,2.000000e-02,0.000000e+00,-0.999999,-0.999999,-1.000000,-1.000000,-1.000000
25%,12156.500000,5.000000,9.800000e+00,1.080700e+06,-0.215759,-0.217422,-0.279690,-0.329617,-0.247802
50%,24313.000000,14.000000,2.220309e+01,5.729400e+06,0.035616,0.031754,0.275402,0.192546,0.617488
75%,36469.500000,25.000000,4.773772e+01,2.125030e+07,0.298387,0.288889,1.037624,0.934480,2.068930
max,48626.000000,61.000000,1.373400e+12,1.197745e+10,198.999997,198.999997,278.416663,649.666696,975.000045


In [17]:
# Make sure types are correct
prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3747507719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3747507719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_1748/3747507719.py:4: SettingWithCopyWarning: 
A value is trying to be set on a 

# Create Dim table

In [18]:
columns_dim = columns['columns_dim']
dim_df = financial[columns_dim]

In [19]:
dim_grouped_df = dim_df.groupby('ticker').first().reset_index()

# Save dataframes to load

In [20]:
ratios_df.to_csv("../../data/load_04/ratios_df.csv")
income_statememnt_df.to_csv("../../data/load_04/income_statememnt_df.csv")
balance_sheet_df.to_csv("../../data/load_04/balance_sheet_df.csv")
cash_flow_statement_df.to_csv("../../data/load_04/cash_flow_statement_df.csv")
prices_df.to_csv("../../data/load_04/prices_df.csv")
dim_grouped_df.to_csv("../../data/load_04/dim_grouped_df.csv")